# Importing

In [5]:
import pandas as pd
import numpy as np
import joblib
import os

# for waterfall plot
import shap
import matplotlib
import matplotlib.pyplot as plt
import dataclasses
from typing import Union
import matplotlib.patches as mpatches

# for load the data pre-preparation function
import importlib.util


In [3]:
np.set_printoptions(suppress=False)

work_dir = r'C:\Users\krasavica\Desktop\Projekty - DS\python-project-ApartmentPriceAnalysis'
os.chdir(work_dir)

pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
X = pd.read_csv("3. Feature importance and report/data_X.csv", index_col = 0)
y = pd.read_csv("3. Feature importance and report/data_y.csv", index_col = 0)

In [13]:
y

,price
0,415000.00
1,880000.00
2,590000.00
3,699000.00
4,1515800.00
...,...
46776,512670.00
46777,599000.00
46778,795515.00
46779,921060.00
